In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
from glob import glob
f_names = {}
for filename in glob('../data/**/*.JPG', recursive=True):
    f_names[filename.split('/')[-1]] = {'path': filename}

markup_updated = pd.read_json('../artifacts/markup_updated.json').T.to_dict()

areas = []
rels = []
for f in markup_updated:
    for m in markup_updated[f]['coco']:
        areas.append(m[-1] * m[-2])
        if m[-1] >= m[-2]:
            rels.append(m[-1]/m[-2])
        else:
            rels.append(m[-2]/m[-1])

In [3]:
# инферим обученную модель на всех картинках
import sys
sys.path.append('../utils/')
from metrics import calc_tp_fp_fn, print_main_metrics, get_true_boxes_format, get_predicted_boxes_format
import joblib

sets_info = joblib.load('../artifacts/sets_info_v3.joblib')
save_path = 'preds_all'
results = joblib.load('../artifacts/res_8s_slicing_0.45.joblib')

fname2index = {j:i for i,j in dict(enumerate(f_names.keys())).items()}
index2fname = {j:i for i,j in fname2index.items()}

/opt/homebrew/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/homebrew/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <9DBE5D5C-AC87-30CA-96DA-F5BC116EDA2B> /opt/homebrew/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <44DEDA27-4DE9-3D4A-8EDE-5AA72081319F> /opt/homebrew/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [32]:
iou_threshold=0.0

detections = get_predicted_boxes_format({i: results[i] for i in results}, fname2index, 
                                        min_area=np.quantile(areas, 0.02), max_area=np.quantile(areas, 0.98))
detections = [i for i in detections if i[2] > 0.45]
ground_truths = get_true_boxes_format({i: markup_updated[i] for i in markup_updated}, fname2index)

tp_list, fp_list, fn = calc_tp_fp_fn(ground_truths, detections, iou_threshold)

ground_truths[:5]

[[423, 0, 1, 692, 810, 764, 852],
 [421, 0, 1, 3751, 218, 3799, 285],
 [284, 0, 1, 6705, 276, 6765, 361],
 [2281, 0, 1, 6470, 2920, 6510, 2954],
 [2281, 0, 1, 5466, 3137, 5489, 3160]]

In [33]:
print(len(tp_list), len(fp_list), fn)

314 4097 27


In [34]:
import joblib

sets_info = joblib.load('../artifacts/sets_info_v3.joblib')

In [35]:
train_names = set(sets_info['train']['names'])
val_names = set(sets_info['val']['names'])

In [36]:
train_tp_list = []
val_tp_list = []
test_tp_list = []
for bbox in tp_list:
    name = index2fname[bbox[0]]
    if name in train_names:
        train_tp_list.append(bbox)
    elif name in val_names:
        val_tp_list.append(bbox)
    else:
        test_tp_list.append(bbox)

train_fp_list = []
val_fp_list = []
test_fp_list = []
for bbox in fp_list:
    name = index2fname[bbox[0]]
    if name in train_names:
        train_fp_list.append(bbox)
    elif name in val_names:
        val_fp_list.append(bbox)
    else:
        test_fp_list.append(bbox)

In [37]:
print('Train dataset:', len(train_tp_list) + len(train_fp_list))
print('Train dolphins:', len(train_tp_list))
print('Train not dolphins:', len(train_fp_list))
print()
print('Val dataset:', len(val_tp_list) + len(val_fp_list))
print('Val dolphins:', len(val_tp_list))
print('Val not dolphins:', len(val_fp_list))
print()
print('Test dataset:', len(test_tp_list) + len(test_fp_list))
print('Test dolphins:', len(test_tp_list))
print('Test not dolphins:', len(test_fp_list))

Train dataset: 261
Train dolphins: 220
Train not dolphins: 41

Val dataset: 104
Val dolphins: 94
Val not dolphins: 10

Test dataset: 4046
Test dolphins: 0
Test not dolphins: 4046


# Выгрузка

In [5]:
import os
import shutil
from pathlib import Path

dataset_name = '../dolphin_postprocces_dataset'

shutil.rmtree(dataset_name ,ignore_errors=True)
Path(dataset_name).mkdir(parents=True, exist_ok=True)

for class_name in ['0', '1']:
    Path(os.path.join(dataset_name, class_name)).mkdir(parents=True, exist_ok=True)
    

# генерация датасета
for class_name, bboxes in [('1', tp_list), ('0', fp_list)]:
    for i, obj in tqdm(enumerate(bboxes)):
        
        img = cv2.imread(f_names[index2fname[obj[0]]]['path'])
        
        x_min, y_min, x_max, y_max = [int(i) for i in obj[3:]]

        crop_img = img[y_min:y_max, x_min:x_max]
        
        cv2.imwrite(os.path.join(dataset_name, class_name, str(i) + '_' +  index2fname[obj[0]]), crop_img)


fnamedataset2ind = {}
fname2confidence = {}
for class_name, bboxes in [('1', tp_list), ('0', fp_list)]:
    for i, obj in tqdm(enumerate(bboxes)):
        fnamedataset2ind[os.path.join(dataset_name, class_name, str(i) + '_' +  index2fname[obj[0]])] = obj[0]
        fname2confidence[os.path.join(dataset_name, class_name, str(i) + '_' +  index2fname[obj[0]])] = obj[2]
        
ind2fnamedataset = {j:i for i,j in fnamedataset2ind.items()}

306it [01:36,  3.17it/s]
336it [01:48,  3.11it/s]


KeyboardInterrupt: 

In [ ]:
!cp -r ../dolphin_postprocces_dataset ../dolphin_postprocces_dataset_full

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image


img_size = 112
size = (img_size, img_size)

class DolphinClassificationDataset(Dataset):

    def __init__(self, dataset: list, transform=None):
        self.img_paths = [i['img_path'] for i in dataset]
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):

        img_name = self.dataset[idx]['img_path']
        image = Image.open(img_name)
        
        
        new_size = (img_size, img_size)
        new_im = Image.new("RGB", new_size)
        box = tuple((n - o) // 2 for n, o in zip(new_size, image.size[:2]))
        new_im.paste(image, box)
        

        # shape = image.size[:2]
        label = int(self.dataset[idx]['class'])
        # conf = self.dataset[idx]['confidence']

        if self.transform:
            new_im = self.transform(new_im)

        return new_im, label #, conf, shape[0], shape[1]


train_transform = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((img_size, img_size)),
                                      transforms.RandomHorizontalFlip(0.5),
                                      transforms.RandomVerticalFlip(0.5),
                                      transforms.RandomRotation(degrees=30),
                                ])

val_transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((img_size, img_size))
                                ])

dataset = []

for class_name in os.listdir(dataset_name):
    for img_name in os.listdir(os.path.join(dataset_name, class_name)):
        dataset.append({'class': class_name,
                        'img_path': os.path.join(dataset_name, class_name, img_name),
                        'confidence': fname2confidence[os.path.join(dataset_name, class_name, img_name)]})
        
dataset[:2]

In [ ]:
!mkdir ../dolphin_postprocces_dataset/test
!mkdir ../dolphin_postprocces_dataset/train
!mkdir ../dolphin_postprocces_dataset/val
!mkdir ../dolphin_postprocces_dataset/train/1
!mkdir ../dolphin_postprocces_dataset/val/1
!mkdir ../dolphin_postprocces_dataset/train/0
!mkdir ../dolphin_postprocces_dataset/val/0

In [ ]:
!mv ../dolphin_postprocces_dataset/1 ../dolphin_postprocces_dataset/test/1
!mv ../dolphin_postprocces_dataset/0 ../dolphin_postprocces_dataset/test/0

In [ ]:
import os
import shutil

directories = ['../dolphin_postprocces_dataset/test/1', '../dolphin_postprocces_dataset/test/0']

for directory in directories:
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        if os.path.isfile(f):
            if f[-12:] in train_names:
                shutil.move(f, f.replace('test', 'train'))
            elif f[-12:] in val_names:
                shutil.move(f, f.replace('test', 'val'))

In [ ]:
print('Train dataset:', len(os.listdir('../dolphin_postprocces_dataset/train/1')) +
      len(os.listdir('../dolphin_postprocces_dataset/train/0')))
print('Train dolphins:', len(os.listdir('../dolphin_postprocces_dataset/train/1')))
print('Train not dolphins:', len(os.listdir('../dolphin_postprocces_dataset/train/0')))
print()
print('Val dataset:', len(os.listdir('../dolphin_postprocces_dataset/val/1')) +
      len(os.listdir('../dolphin_postprocces_dataset/val/0')))
print('Val dolphins:', len(os.listdir('../dolphin_postprocces_dataset/val/1')))
print('Val not dolphins:', len(os.listdir('../dolphin_postprocces_dataset/val/0')))
print()
print('Test dataset:', len(os.listdir('../dolphin_postprocces_dataset/test/1')) +
      len(os.listdir('../dolphin_postprocces_dataset/test/0')))
print('Test dolphins:', len(os.listdir('../dolphin_postprocces_dataset/test/1')))
print('Test not dolphins:', len(os.listdir('../dolphin_postprocces_dataset/test/0')))

In [ ]:
print('Full dataset:', len(os.listdir('../dolphin_postprocces_dataset_full/1')) +
      len(os.listdir('../dolphin_postprocces_dataset_full/0')))
print('Full dolphins:', len(os.listdir('../dolphin_postprocces_dataset_full/1')))
print('Full not dolphins:', len(os.listdir('../dolphin_postprocces_dataset_full/0')))
print()

In [ ]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(dataset, test_size=0.3, random_state=42)

train_set = DolphinClassificationDataset(train, train_transform)
val_set = DolphinClassificationDataset(val, val_transform)

print(len(train_set), len(val_set))

batch_size = 64

train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
valid_dataloader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=0)